### 1. Просьба прислать код (Python), который бы предсказывал целевые переменные - X3 , Y3 и VALUE (фокус на ETL, data transformation)

In [1]:
import pandas as pd

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20180916023344-0001


In [2]:
task = pd.read_csv("https://www.dropbox.com/s/q5tg88psu8k1yuz/task.csv?dl=1")

In [3]:
task.head()

,report_dt,ID,1.1.,1.2.,1.3.,1.4.,1.5.,1.6.,1.7.,1.8.,...,3.9.,3.10.,3.11.,3.12.,3.13.,3.14.,3.15.,X3,Y3,VALUE
0,30.04.2013,1-15JF8Z,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,31.07.2013,1-6IW9T,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,19.0
2,30.04.2012,1-2712QP,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
3,30.09.2013,1-121YJOD,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,31.01.2014,1-12667,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,5.0


In [4]:
task.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42602 entries, 0 to 42601
Data columns (total 55 columns):
report_dt    42602 non-null object
ID           42602 non-null object
1.1.         42602 non-null int64
1.2.         42602 non-null int64
1.3.         42602 non-null int64
1.4.         42602 non-null int64
1.5.         42602 non-null int64
1.6.         42602 non-null int64
1.7.         42602 non-null int64
1.8.         42602 non-null int64
1.9.         42602 non-null int64
1.10.        42602 non-null int64
1.11.        42602 non-null int64
1.12.        42602 non-null int64
1.13.        42602 non-null int64
1.14.        42602 non-null int64
1.15.        42602 non-null int64
1.16.        42602 non-null int64
1.17.        42602 non-null int64
1.18.        42602 non-null int64
1.19.        42602 non-null int64
2.1.         42602 non-null int64
2.2.         42602 non-null int64
2.3.         42602 non-null int64
2.4.         42602 non-null int64
2.5.         42602 non-null int64
2.6. 

In [5]:
len(task["ID"].unique())

1599

In [6]:
len(task["report_dt"].unique())

45

In [7]:
len(task[task["X3"].notnull() & task["Y3"].notnull() & task["VALUE"].notnull()])

31083

Фрейм содержит 42602 строки  
31083 строки с непустыми значениями X3, Y3, VALUE  
Во фрейме 1599 уникальных ID и 45 уникальных report_dt  
Значения X3, Y3 бинарные (0 или 1)  
Значения VALUE числовые  
Средние значения VALUE для разных ID могут отличаться на порядки. Внутри одного ID значения VALUE более равномерны.  
Проведем нормализацию значений VALUE внутри групп ID (z-score)

In [8]:
t = task[task["X3"].notnull() & task["Y3"].notnull() & task["VALUE"].notnull()]

In [9]:
avg = t.groupby('ID').mean()

In [10]:
std = t.groupby('ID').std()

In [11]:
t['VALUEAVG'] = t.join(avg, on = 'ID', lsuffix = 't', rsuffix = 'AVG')['VALUEAVG']

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
t['VALUESTD'] = t.join(std, on = 'ID', lsuffix = 't', rsuffix = 'STD')['VALUESTD']

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
t['VALUENEW'] = (t['VALUE'] - t['VALUEAVG'])/(t['VALUESTD'])

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
len(t[t['VALUENEW'].isnull()])

52

Для 52 наблюдений z-score не расчитался, из-за того что внутри соответствующих ID одно наблюдение или два наблюдения с одинаковыми значениями. Такие наблюдения исключаем из выборки

In [15]:
t1 = t.dropna(subset = ["VALUENEW"])

In [16]:
x_data = t1.drop(['ID', 'report_dt', 'X3', 'Y3', 'VALUE', 'VALUEAVG', 'VALUESTD', 'VALUENEW'], axis = 1)

In [17]:
y_data = t1[['X3', 'Y3', 'VALUENEW']]

разделим выборку на обучающую и тестовую в отношении 70/30

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3, random_state = 0)

Для предсказания бинарных переменных X3, Y3 испольлзуем логистическую регрессию,   
для предсказания переменной VALUE - линейную регрессию

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

In [21]:
X3reg = LogisticRegression()
Y3reg = LogisticRegression()
VALUEreg = LinearRegression()

In [22]:
X3reg.fit(x_train, y_train[['X3']])

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
Y3reg.fit(x_train, y_train[['Y3']])

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
VALUEreg.fit(x_train, y_train[['VALUENEW']])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

### 2. Подберите и обоснуйте выбор метрики для оценки точности разработанных моделей. 

Для оценки точности линейной модели посчитаем R^2 для обучающей и тестовой выборок:

In [25]:
print("Переменная VALUE, R^2 на обучающей выборке: ", VALUEreg.score(x_train, y_train[['VALUENEW']]), "\nR^2 на тестовой выборке:", VALUEreg.score(x_test, y_test[['VALUENEW']]))

Переменная VALUE, R^2 на обучающей выборке:  0.0669943522243 
R^2 на тестовой выборке: 0.0620903293588


Значение R^2 близко к 0, это говорит о том что модель не подходит для предсказания значения переменной VALUE

Для бинарных переменных X3, Y3 рассчитаем метрики precision, recall и F1-score

In [26]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [27]:
print("X3 precision score: ", precision_score(y_test[['X3']], X3reg.predict(x_test)), "\nX3 recall score: ", recall_score(y_test[['X3']], X3reg.predict(x_test)), 
      '\nX3 F1 score: ', f1_score(y_test[['X3']], X3reg.predict(x_test)))

X3 precision score:  0.745945945946 
X3 recall score:  0.46 
X3 F1 score:  0.569072164948


In [28]:
print("Y3 precision score: ", precision_score(y_test[['Y3']], Y3reg.predict(x_test)), "\nY3 recall score: ", recall_score(y_test[['Y3']], Y3reg.predict(x_test)), 
      '\nY3 F1 score: ', f1_score(y_test[['Y3']], Y3reg.predict(x_test)))

Y3 precision score:  0.789473684211 
Y3 recall score:  0.661764705882 
Y3 F1 score:  0.72


Сохраним модели в файлы (pickle) для дальнейшего использования в микросервисе

In [29]:
# @hidden_cell
credentials_1 = {
  'bucket':'tasksberbank-donotdelete-pr-rkzvlwvaljtg6z',
  'secret_key':'8d3e5ed483f3ae3f24c58ba05938f46f4e36f4cfbc990c7d',
  'iam_url':'https://iam.eu-gb.bluemix.net/oidc/token',
  'api_key':'ftrmNH3RV0j1NinLwcYkCbDkBnRcxi84zfoqe-K0nQtg',
  'resource_instance_id':'crn:v1:bluemix:public:cloud-object-storage:global:a/33ffbd73ce894777adc04907e1c9ff18:74545dad-5f02-4a45-b6cf-6bc12ca4c36d:bucket:tasksberbank-donotdelete-pr-rkzvlwvaljtg6z',
  'access_key':'25cbf6ee58924dba925516894d212b7f',
  'url':'https://s3.eu-geo.objectstorage.service.networklayer.com'
}

In [30]:
from ibm_botocore.client import Config
import ibm_boto3
cos = ibm_boto3.client('s3',
                    ibm_api_key_id=credentials_1['api_key'],                    
                    ibm_service_instance_id=credentials_1['resource_instance_id'],
                    ibm_auth_endpoint=credentials_1['iam_url'],
                    config=Config(signature_version='oauth'),
                    endpoint_url=credentials_1['url'])

In [31]:
import pickle

In [32]:
pickle.dump(X3reg, open("X3reg.pkl", 'wb'))
cos.upload_file('X3reg.pkl', credentials_1['bucket'],'X3reg.pkl')

In [33]:
pickle.dump(Y3reg, open("Y3reg.pkl", 'wb'))
cos.upload_file('Y3reg.pkl', credentials_1['bucket'],'Y3reg.pkl')

In [34]:
pickle.dump(VALUEreg, open("VALUEreg.pkl", 'wb'))
cos.upload_file('VALUEreg.pkl', credentials_1['bucket'],'VALUEreg.pkl')